In [2]:
import pandas as pd
import numpy as np

# demand-forecasting-kernels-only

In [13]:
train = pd.read_csv(r"data/demand-forecasting-kernels-only/train.csv")
train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [14]:
train.date.min(),train.date.max()

('2013-01-01', '2017-12-31')

In [4]:
test = pd.read_csv(r"data/demand-forecasting-kernels-only/test.csv")
test

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [12]:
test_min_date, test_max_date = test.date.unique().min(),test.date.unique().max()
test_min_date, test_max_date

('2018-01-01', '2018-03-31')

In [225]:
df = pd.concat([train, test], axis=0)
df0 = df.copy(deep=True)
df

,date,store,item,sales,id
0,2013-01-01,1,1,13.0,NaN
1,2013-01-02,1,1,11.0,NaN
2,2013-01-03,1,1,14.0,NaN
3,2013-01-04,1,1,13.0,NaN
4,2013-01-05,1,1,10.0,NaN
...,...,...,...,...,...
44995,2018-03-27,10,50,NaN,44995.0
44996,2018-03-28,10,50,NaN,44996.0
44997,2018-03-29,10,50,NaN,44997.0
44998,2018-03-30,10,50,NaN,44998.0


## User configured columns

In [255]:
time_col = 'date'
index_cols = ['store','item']
target = 'sales'
target_fillna_value = 0

In [48]:
df[1:]

In [165]:
df[1:][time_col].drop_duplicates().sort_values().reset_index(drop=True)

0       2013-01-01
1       2013-01-02
2       2013-01-03
3       2013-01-04
4       2013-01-05
           ...    
1911    2018-03-27
1912    2018-03-28
1913    2018-03-29
1914    2018-03-30
1915    2018-03-31
Name: date, Length: 1916, dtype: object

In [106]:
df[1:][index_cols].drop_duplicates()

,store,item
1,1,1
1826,2,1
3652,3,1
5478,4,1
7304,5,1
...,...,...
903870,6,50
905696,7,50
907522,8,50
909348,9,50


In [110]:
index_col = "_".join(index_cols)

'store_item'

In [284]:
from itertools import product
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator

def move_cols_to_first(the_df, first_cols: list):
    latter_cols = the_df.columns[~the_df.columns.isin(first_cols)]
    the_df = pd.concat([the_df[first_cols], the_df[latter_cols]],axis=1)
    return the_df

def add_concat_index_cols(the_df):
    indexes_col = "_".join(index_cols)
    the_df[indexes_col] = the_df[index_cols[0]].astype(str)
    if len(indexes_col)>1:
        for col in index_cols[1:]:
            the_df[indexes_col] = the_df[indexes_col] + "_" + the_df[col].astype(str)
    the_df = move_cols_to_first(the_df, [time_col,indexes_col]+index_cols)
    return the_df, indexes_col

def is_full_time_df(the_df):
    is_full_time = len(the_df) == reduce(operator.mul, the_df[index_cols+[time_col]].nunique())
    return is_full_time

def fillna_target(the_df):
    assert target in the_df
    the_df = the_df.fillna({target:target_fillna_value})
    assert the_df[target].isna().sum() == 0
    return the_df

def add_missing_dates_fill_target_zero(the_df):
    if is_full_time_df(the_df):
        return the_df
    # Make sure there's one index col
    the_indexes_col = "_".join(index_cols)
    if not the_indexes_col in the_df:
        the_df, the_indexes_col = add_concat_index_cols(the_df)
    # The logic
    the_df_full_date_indexes = pd.DataFrame(product(the_df[time_col].drop_duplicates(), the_df[the_indexes_col].drop_duplicates()), columns=[time_col, the_indexes_col]).dropna()
    the_df = the_df.merge(the_df_full_date_indexes, how='right')
    the_df = fillna_target(the_df)
    assert is_full_time_df(the_df)
    return the_df

add_missing_dates_fill_target_zero(df0[1:])

,date,store,item,sales,id,store_item
0,2013-01-02,1.0,1.0,11.0,NaN,1_1
1,2013-01-03,1.0,1.0,14.0,NaN,1_1
2,2013-01-04,1.0,1.0,13.0,NaN,1_1
3,2013-01-05,1.0,1.0,10.0,NaN,1_1
4,2013-01-06,1.0,1.0,12.0,NaN,1_1
...,...,...,...,...,...,...
957995,2018-03-28,10.0,50.0,0.0,44996.0,10_50
957996,2018-03-29,10.0,50.0,0.0,44997.0,10_50
957997,2018-03-30,10.0,50.0,0.0,44998.0,10_50
957998,2018-03-31,10.0,50.0,0.0,44999.0,10_50


# TODO
## create a unified time col from multiple time cols